<a href="https://colab.research.google.com/github/zahraDehghanian97/LSH_Spark/blob/master/Question3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# تمرین سری سوم درس پردازش داده های حجیم
# زهرا دهقانیان
# 401300417




In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq --fix-missing
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 16.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=f39bd88097f5d41477ddf9f19ca96a0313faba1ccabec6aa0c3bc92d914cc710
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The followin

In [2]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [4]:
from google.colab import drive
# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
from pyspark import SparkContext, SparkConf 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,TimestampType
from pyspark.sql.functions import col,current_timestamp,to_date,hour,dayofweek
spark = SparkSession \
    .builder \
    .appName("Fave_Project") \
    .master("local[*]") \
    .getOrCreate()

sc=spark.sparkContext

schema = StructType([ \
        StructField("DEVICE_CODE", IntegerType(), True), 
        StructField("SYSTEM_ID",IntegerType(),True), \
        StructField("ORIGINE_CAR_KEY",StringType(),True), \
        StructField("FINAL_CAR_KEY",StringType(),True), \
        StructField("CHECK_STATUS_KEY", IntegerType(), True), \
        StructField("COMPANY_ID", StringType(), True), \
        StructField("PASS_DAY_TIME", TimestampType(), True)
    ])

In [7]:
df=spark.read.csv('/content/drive/MyDrive/SampleData.csv',header=True,schema=schema)
df.show(1)

+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|DEVICE_CODE|SYSTEM_ID|ORIGINE_CAR_KEY|FINAL_CAR_KEY|CHECK_STATUS_KEY|COMPANY_ID|      PASS_DAY_TIME|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
|  100700853|      283|        9016704|      9016704|               5|       264|2022-01-08 00:00:39|
+-----------+---------+---------------+-------------+----------------+----------+-------------------+
only showing top 1 row



الف) مسیر عبوری هر خودرو را به تفکیک روز مشخص کنید 

In [8]:
df_mapped = df.rdd.map(lambda x: ((x['ORIGINE_CAR_KEY'], x['PASS_DAY_TIME'].date()), [x['DEVICE_CODE']]))
df_mapped3 = df_mapped.reduceByKey(lambda a,b: a+b)
df_mapped2 = df_mapped3   #.sample(False,0.1)
df_mapped2.take(10)

[(('9017538', datetime.date(2022, 1, 8)), [230204, 631357]),
 (('9035020', datetime.date(2022, 1, 8)), [900164]),
 (('9020900', datetime.date(2022, 1, 8)), [900149]),
 (('9021792', datetime.date(2022, 1, 8)),
  [900246,
   100700871,
   900235,
   631697,
   22009971,
   900265,
   900235,
   100700871,
   22009972,
   900215,
   209103]),
 (('9038947', datetime.date(2022, 1, 8)),
  [100701059, 900234, 900234, 100701059, 117, 117, 117]),
 (('9024221', datetime.date(2022, 1, 8)), [631765]),
 (('9012923', datetime.date(2022, 1, 8)), [900101, 100700841]),
 (('9009732', datetime.date(2022, 1, 8)), [900227]),
 (('9031414', datetime.date(2022, 1, 8)), [900191, 900129, 900275]),
 (('9031445', datetime.date(2022, 1, 8)), [100701092, 22010053])]

ب)محاسبه شباهت کسینوسی بین یک مسیر فرضی و مسیر خودروها و گزارش پنج مسیر و خودرو با بیشترین شباهت  

برای یافتن فاصله‌ی کسینوسی به جای تشکیل بردار با طول هزار متشکل از مقادیر صفر و یک برای هر مسیر میتوان تعداد کدهای مشترک بین هر مسیر و مسیر فرضی را بدست آورد و سپس بر اندازه‌ی بردار مسیر پیشنهادی و همچنین اندازه‌ی مسیر هر خودرو تقسیم نمود.
 در این مساله خروجی این روش همان خروجی موردنظر یعنی شباهت کسینوسی خواهد بود و فقط بسیار ساده تر محاسبه می شود

In [34]:
import math 
path = [900246,100700871,900235,631697] #masire farzi
lenPath = 10
similarity_rdd = df_mapped2.map(lambda x: ((x[0][0], x[1]), len(list(set(path) & set(x[1])))/math.sqrt(len(x[1])*lenPath)))
similarity_rdd.take(20)

[(('9017538', [230204, 631357]), 0.0),
 (('9035020', [900164]), 0.0),
 (('9020900', [900149]), 0.0),
 (('9021792',
   [900246,
    100700871,
    900235,
    631697,
    22009971,
    900265,
    900235,
    100700871,
    22009972,
    900215,
    209103]),
  0.38138503569823695),
 (('9038947', [100701059, 900234, 900234, 100701059, 117, 117, 117]), 0.0),
 (('9024221', [631765]), 0.0),
 (('9012923', [900101, 100700841]), 0.0),
 (('9009732', [900227]), 0.0),
 (('9031414', [900191, 900129, 900275]), 0.0),
 (('9031445', [100701092, 22010053]), 0.0),
 (('9016898', [900212, 900269]), 0.0),
 (('9026982', [900266]), 0.0),
 (('9024336', [900164, 100700880, 100700880, 900138]), 0.0),
 (('9024374',
   [22009972, 900215, 900251, 103002, 900234, 631829, 900251, 631366]),
  0.0),
 (('9136608',
   [202101, 202901, 203902, 203902, 203902, 203902, 900248, 203902]),
  0.0),
 (('9190591', [900174]), 0.0),
 (('9153235', [900142, 900156, 900199]), 0.0),
 (('9150695', [900212, 900224]), 0.0),
 (('9156538'

پس از بدست آمدن مقادیر شباهت کسینوسی هر مسیر با مسیر فرضی ۵ مسیر و خودرو با بیشترین شباهت با مسیر فرضی را نمایش میدهیم

In [35]:
similarPath = similarity_rdd.takeOrdered(5, key = lambda x: -x[1])
similarPath

[(('57373850', [900235, 900246, 100700871]), 0.5477225575051661),
 (('15787221', [900246, 900235, 100700871]), 0.5477225575051661),
 (('92669360', [900235, 900246, 100700871]), 0.5477225575051661),
 (('10055017', [900235, 631765, 900246, 100700871]), 0.4743416490252569),
 (('90966204', [100700871, 900235, 900246, 631357]), 0.4743416490252569)]

پ) با استفاده از روش ال اس اج بخش ب را حل کنید  و محاسبات مربوط به نحوه
استفاده از آن ها را انجام دهید. سپس مسیر های مشابه با مسیر فرضی را گزارش کنید. در صورت افزایش تعداد هایپرپلین ها دقت به چه صورت افزایش می یابد.

برای یافتن بردارهای مشابه با بردار فرضی تعریف شده به کمک روش ال اس اچ، اول بردار دوربینهای مجزا را می سازیم. در این مسیله ابتدا تعداد توابع مین هش یعنی هایپرپلین ها را برابر با 20۰ قرار میدهیم و در ادامه تغییرات را با دوبرابر کردنشان بررسی میکنیم. 

In [70]:
hyperplane = 20 
# f = df_mapped2.collect()
# l = []
# for li in f :
#   l.extend(li)
# allcameradist = list(set(l))
allcamera = df_mapped2.flatMap(lambda line: line[1])
allcameradist = allcamera.distinct().collect()
sc.broadcast(allcameradist)
l = len(allcameradist)
len(allcameradist)

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.8/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: ignored

 برای یافتن بردار هش شده‌ی به طول 20 از هر یک از مسیرها، بر اساس وجود اولین یک در هر یک از  جایگشت رندوم ایجاد شده در بالا، عمل می کنیم. در توابع زیرفرایند تولید جایگشت ها و بررسی وجود دوربین فوق در مسیر انجام میگیرد

In [ ]:
import numpy as np
random_start = []
for i in range(hyperplane):
  random_start.append(np.random.permutation(l))
sc.broadcast(random_start)


In [ ]:

def findfirstone(arr, per, size):
  for i in range(len(arr)):
    if arr[per[i]]==1 :
      return per[i]
  return len(arr)

def minHashing(row):
  x = row[1]
  result = []
  place = np.zeros(len(allcameradist))
  for j in range(len(x)):
      place[allcameradist.index(x[j])] = 1
  place = list(place)
  for i in random_start:
      result.append(findfirstone(place, i, len(allcameradist)))

  return (row[1], result)

df_lsh = df_mapped2.map(minHashing)
df_lsh.take(1)

همین کار را برای مسیر فرضیمان نیز انجام میدهیم و بردار هش شده به طول 20 آن را بدست می‌آوریم

In [ ]:
path = (0,[900246,100700871,900235,631697])
path_lsh = minHashing(path)
sc.broadcast(path_lsh)
path_lsh

پس از بدست آوردن هش هر مسیر آن را به دو صورت با هش مسیر فرضی مقایسه میکنیم



1.   and then or
2.   or then and

در این جا خروجی برای هر مسیر به این صورت می باشد:

(تعداد هش های یکسان تقسیم بر کل تعداد هش ها، حاصل اور، حاصل اند،  مسیر )

r= 5 و b=4 

In [ ]:
r = 5
b = 4
sc.broadcast(r)
sc.broadcast(b)

def check_similarity(x):
  y = x[1]
  orres = list(np.full(b, False))
  andres = list(np.full(b, True))
  sum = 0
  for i in range(len(y)):
    andres[int(i/r)] = andres[int(i/r)] and (path_lsh[i] == y[i])
    orres[int(i/r)] = orres[int(i/r)] or (path_lsh[i] == y[i])
    if(path_lsh[i] == y[i]):
      sum += 1
  return (x[0],(andres,orres,sum/len(y)))

df_similarity_LSH = df_lsh.map(check_similarity)
df_similarity_LSH.take(10)

In [ ]:
def and_then_or(x):
    res = False
    for i in x[1][0]:
      res = res or i
    return (x[0],(x[1][2],res))

and_or_result = df_similarity_LSH.map(and_then_or)

similar_or = and_or_result.filter(lambda x: x[1][1] == True)
similar_or.collect()
similar_or2 = similar_or.takeOrdered(5, key = lambda x: -x[1][0])
similar_or2

In [ ]:
def or_then_and(x):
      res = True
      for i in x[1][1]:
        res = res and i
      return (x[0],(x[1][2],res))

or_and_result = df_similarity_LSH.map(or_then_and)

similar_and = or_and_result.filter(lambda x: x[1][1] == True)
similar_and.collect()
similar_and2 = similar_and.takeOrdered(5, key = lambda x: -x[1][0])
similar_and2


حال همین کار را با تعداد هایپر پلین های 40  انجام میدهیم

r=5 , b=8

In [ ]:
hyperplane = 40
r = 5
b = 8
l = len(allcameradist)
sc.broadcast(r)
sc.broadcast(b)

import numpy as np
random_start = []
for i in range(hyperplane):
  random_start.append(np.random.permutation(l))
sc.broadcast(random_start)

df_lsh2 = df_mapped2.map(minHashing)
path_lsh = minHashing(path)
sc.broadcast(path_lsh)

df_similarity_LSH2 = df_lsh2.map(check_similarity)
df_similarity_LSH2.take(10)

In [ ]:
and_or_result2 = df_similarity_LSH2.map(and_then_or)

similar_or2 = and_or_result2.filter(lambda x: x[1][1] == True)
similar_or2.collect()
similar_or22 = similar_or2.takeOrdered(5, key = lambda x: -x[1][0])
similar_or22

In [ ]:
or_and_result2 = df_similarity_LSH2.map(or_then_and)

similar_and2 = or_and_result2.filter(lambda x: x[1][1] == True)
similar_and2.collect()
similar_and22 = similar_and2.takeOrdered(5, key = lambda x: -x[1][0])
similar_and22

بنظر می رسد با افزایش تعداد هایپر پلین ها، دقت به صورت نمایی بهتر شده است  